# Processing large datasets with Apache Spark and Amazon SageMaker

***This notebook run on `Data Science 3.0 - Python 3` kernel on a `ml.t3.large` instance***.

Amazon SageMaker Processing Jobs are used  to analyze data and evaluate machine learning models on Amazon SageMaker. With Processing, you can use a simplified, managed experience on SageMaker to run your data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation. You can also use the Amazon SageMaker Processing APIs during the experimentation phase and after the code is deployed in production to evaluate performance.

        


![](https://docs.aws.amazon.com/images/sagemaker/latest/dg/images/Processing-1.png)

The preceding diagram shows how Amazon SageMaker spins up a Processing job. Amazon SageMaker takes your script, copies your data from Amazon Simple Storage Service (Amazon S3), and then pulls a processing container. The processing container image can either be an Amazon SageMaker built-in image or a custom image that you provide. The underlying infrastructure for a Processing job is fully managed by Amazon SageMaker. Cluster resources are provisioned for the duration of your job, and cleaned up when a job completes. The output of the Processing job is stored in the Amazon S3 bucket you specified.

## Our workflow for processing large amounts of data with SageMaker

We can divide our workflow into two steps:
    
1. Work with a small subset of the data with Spark running in local model in a SageMaker Studio Notebook.

1. Once we are able to work with the small subset of data we can provide the same code (as a Python script rather than a series of interactive steps) to SageMaker Processing which launched a Spark cluster, runs out code and terminates the cluster.

## In this notebook...

We will analyze the [Pushshift Reddit dataset](https://arxiv.org/pdf/2001.08435.pdf) to be used for the project and then we will run a SageMaker Processing Job to filter out the comments and submissions from subreddits of interest. The filtered data will be stored in your account's s3 bucket and it is this filtered data that you will be using for your project.

## Setup
We need an available Java installation to run pyspark. The easiest way to do this is to install JDK and set the proper paths using conda

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.2.0

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-65923ea2-8789-48f4-8cb4-8507f25e88ff;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 352ms :: artifacts dl 17ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

3.2.0


## Process S3 data with SageMaker Processing Job `PySparkProcessor`

We are going to move the above processing code in a Python file and then submit that file to SageMaker Processing Job's [`PySparkProcessor`](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_processing.html#pysparkprocessor).

In [4]:
!mkdir -p ./code

In [5]:
%%writefile ./code/process.py

import os
import logging
import argparse

# Import pyspark and build Spark session
from pyspark.sql.functions import *
from pyspark.sql.types import (
    DoubleType,
    IntegerType,
    StringType,
    StructField,
    StructType,
)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

logging.basicConfig(format='%(asctime)s,%(levelname)s,%(module)s,%(filename)s,%(lineno)d,%(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

def main():
    parser = argparse.ArgumentParser(description="app inputs and outputs")
    parser.add_argument("--s3_dataset_path", type=str, help="Path of dataset in S3")    
    parser.add_argument("--s3_output_bucket", type=str, help="s3 output bucket")
    parser.add_argument("--s3_output_prefix", type=str, help="s3 output prefix")
    parser.add_argument("--col_name_for_filtering", type=str, help="Name of the column to filter")
    parser.add_argument("--values_to_keep", type=str, help="comma separated list of values to keep in the filtered set")
    args = parser.parse_args()

    spark = SparkSession.builder.appName("PySparkApp").getOrCreate()
    logger.info(f"spark version = {spark.version}")
    
    # This is needed to save RDDs which is the only way to write nested Dataframes into CSV format
    sc = spark.sparkContext
    sc._jsc.hadoopConfiguration().set(
        "mapred.output.committer.class", "org.apache.hadoop.mapred.FileOutputCommitter"
    )

   
    # Downloading the data from S3 into a Dataframe
    logger.info(f"going to read {args.s3_dataset_path}")
    df = spark.read.parquet(args.s3_dataset_path, header=True)
    logger.info(f"finished reading files...")
    

    
    # filter the dataframe to only keep the values of interest
    vals = [s.strip() for s in args.values_to_keep.split(",")]
    df_filtered = df.where(col(args.col_name_for_filtering).isin(vals))
    
    # save the filtered dataframes so that these files can now be used for future analysis
    s3_path = f"s3://{args.s3_output_bucket}/{args.s3_output_prefix}"
    logger.info(f"going to write data for {vals} in {s3_path}")
    logger.info(f"shape of the df_filtered dataframe is {df_filtered.count():,}x{len(df_filtered.columns)}")
    df_filtered.write.mode("overwrite").parquet(s3_path)
    
    logger.info(f"all done...")
    
if __name__ == "__main__":
    main()

Writing ./code/process.py


Now submit this code to SageMaker Processing Job.

In [6]:
%%time
import time
import sagemaker
from sagemaker.spark.processing import PySparkProcessor

# Setup the PySpark processor to run the job. Note the instance type and instance count parameters. SageMaker will create these many instances of this type for the spark job.
role = sagemaker.get_execution_role()
spark_processor = PySparkProcessor(
    base_job_name="sm-spark-project",
    framework_version="3.3",
    role=role,
    instance_count=8,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=3600,
)

# s3 paths
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_logs = f"spark_logs"

# modify this comma separated list to choose the subreddits of interest
subreddits = "MakeupAddiction"
configuration = [
    {
        "Classification": "spark-defaults",
        "Properties": {"spark.executor.memory": "12g", "spark.executor.cores": "4"},
    }
]

# the dataset contains data for these 3 years
year_list = [2021,2022,2023]

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
CPU times: user 2.68 s, sys: 340 ms, total: 3.02 s
Wall time: 3.28 s


In [7]:
%%time
for yyyy in year_list:
    print(f"going to filter comments data for year={yyyy}")
    s3_dataset_path_commments = f"s3://bigdatateaching/reddit-parquet/comments/year={yyyy}/month=*/*.parquet" # "s3a://bigdatateaching/reddit/parquet/comments/yyyy=*/mm=*/*comments*.parquet"
    output_prefix_data_comments = f"project/comments/yyyy={yyyy}"
    col_name_for_filtering = "subreddit"
    subreddits = "MakeupAddiction"

    # run the job now, the arguments array is provided as command line to the Python script (Spark code in this case).
    spark_processor.run(
        submit_app="./code/process.py",
        arguments=[
            "--s3_dataset_path",
            s3_dataset_path_commments,
            "--s3_output_bucket",
            bucket,
            "--s3_output_prefix",
            output_prefix_data_comments,
            "--col_name_for_filtering",
            col_name_for_filtering,
            "--values_to_keep",
            subreddits,
        ],
        spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, output_prefix_logs),
        logs=False,
        configuration=configuration
    )
    # give some time for resources from this iterations to get cleaned up
    # if we start the job immediately we could get insufficient resources error
    time.sleep(60)

going to filter comments data for year=2021


INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-12-05-05-27-25-071


.........................................................................................................................................................................................................!

INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-12-05-05-45-23-990


going to filter comments data for year=2022
......................................................................................................................................................................................................................!

INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-12-05-06-04-27-827


going to filter comments data for year=2023
.......................................................................................................................!CPU times: user 2.52 s, sys: 379 ms, total: 2.9 s
Wall time: 48min 8s


In [8]:
year_list = [2021,2022,2023]

In [9]:
%%time
for yyyy in year_list:
    print(f"going to filter submissions data for year={yyyy}")
    s3_dataset_path_submissions = f"s3://bigdatateaching/reddit-parquet/submissions/year={yyyy}/month=*/*.parquet" # "s3a://bigdatateaching/reddit/parquet/submissions/yyyy=*/mm=*/*submissions*.parquet"
    output_prefix_data_submissions = f"project/submissions/yyyy={yyyy}"

    col_name_for_filtering = "subreddit"
    subreddits = "MakeupAddiction"
    # run the job now, the arguments array is provided as command line to the Python script (Spark code in this case).
    spark_processor.run(
        submit_app="./code/process.py",
        arguments=[
             "--s3_dataset_path",
            s3_dataset_path_submissions,
            "--s3_output_bucket",
            bucket,
            "--s3_output_prefix",
            output_prefix_data_submissions,
            "--col_name_for_filtering",
            col_name_for_filtering,
            "--values_to_keep",
            subreddits,
        ],
        spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, output_prefix_logs),
        logs=False,
        configuration=configuration
    )
    # give some time for resources from this iterations to get cleaned up
    # if we start the job immediately we could get insufficient resources error
    time.sleep(60)

going to filter submissions data for year=2021


INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-12-05-06-15-33-523


...................................................................................................................................................................................................................................................!

INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-12-05-06-37-03-432


going to filter submissions data for year=2022
...........................................................................................................................................................................................................................................................!

INFO:sagemaker:Creating processing-job with name sm-spark-project-2023-12-05-06-59-13-642


going to filter submissions data for year=2023
................................................................................................................................................!CPU times: user 2.9 s, sys: 459 ms, total: 3.36 s
Wall time: 56min 51s


## Read the filtered data

Now that we have filtered the data to only keep submissions and comments from subreddits of interest. Let us read data from the s3 path where we saved the filtered data.

In [21]:
%%time
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_data_comments = "project/comments/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
#s3_path = "s3a://sagemaker-us-east-1-038932893404/project/comments/yyyy=2021/part-00000-90796409-5783-4705-92c0-27c27eda8c4c-c000.snappy.parquet"
print(f"reading comments from {s3_path}")
comments = spark.read.parquet(s3_path, header=True)
print(f"shape of the comments dataframe is {comments.count():,}x{len(comments.columns)}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading comments from s3a://sagemaker-us-east-1-834346005548/project/comments/yyyy=*


shape of the comments dataframe is 1,053,280x21
CPU times: user 353 ms, sys: 52.7 ms, total: 406 ms
Wall time: 5min 20s


In [22]:
# check counts (ensuring all needed subreddits exist)
comments.groupBy('subreddit').count().show()

+---------------+-------+
|      subreddit|  count|
+---------------+-------+
|MakeupAddiction|1053280|
+---------------+-------+



In [23]:
# display a subset of columns
comments.select("subreddit", "author", "body", "parent_id", "link_id", "id", "created_utc").show()

+---------------+--------------------+--------------------+----------+---------+-------+-------------------+
|      subreddit|              author|                body| parent_id|  link_id|     id|        created_utc|
+---------------+--------------------+--------------------+----------+---------+-------+-------------------+
|MakeupAddiction|        chinghiskhan|  Aww, thank you! 💙|t1_hs5p1jj|t3_s125dy|hs5vt5h|2022-01-11 04:04:19|
|MakeupAddiction|        chinghiskhan|Aw thanks, that’s...|t1_hs5q3g2|t3_s125dy|hs5vv80|2022-01-11 04:04:46|
|MakeupAddiction|        chinghiskhan|Omg thank you so ...|t1_hs5r131|t3_s125dy|hs5vwrf|2022-01-11 04:05:04|
|MakeupAddiction|        ssuperficial|Too Faced had a k...| t3_s0u3nh|t3_s0u3nh|hs5wdjb|2022-01-11 04:08:26|
|MakeupAddiction|Equivalent-Newt-4592|All the eyeshadow...|t1_hs5p5dn|t3_s11hyr|hs5wh9u|2022-01-11 04:09:07|
|MakeupAddiction|           ceklassen|Beautiful colours...| t3_s125dy|t3_s125dy|hs5wkwp|2022-01-11 04:09:46|
|MakeupAddiction|   

In [24]:
comments.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- is_submitter: boolean (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)



Sample the dataset around 10000 rows

In [25]:
print(f"Total number of rows: {comments.count()}")
print(f"Total number of unique authors: {comments.select('author').distinct().count()}")
print(f"Total number of unique subreddits: {comments.select('subreddit').distinct().count()}")

Total number of rows: 1053280


Total number of unique authors: 193808


Total number of unique subreddits: 1


In [10]:
from pyspark.sql.functions import col, length
# Check for missing values in all columns
def check_missing_values(df):
    for column in df.columns:
        missing_values = df.filter(col(column).isNull()).count()
        if missing_values > 0:
            print(f"Column {column} has {missing_values} missing values")
        else:
            print(f"Column {column} has no missing values")

In [16]:
# Check for rows with very short comments, which may not be useful for analysis
def filter_short_comments(df, min_length=10):
    return df.filter(length(col("comment")) >= min_length)

# Remove corrupted rows, here we define 'corrupted' rows as examples, it can include any row that does not meet your data criteria
def remove_corrupted_rows(df):
    # we remove rows where 'user_id' is negative or null as an example
    df = df.filter(col("user_id").isNotNull() & (col("user_id") > 0))
    # Add more filters based on your data and needs
    return df

# Apply the functions to check and clean the dataset
def clean_data(df):
    check_missing_values(df)
    df = filter_short_comments(df)
    df = remove_corrupted_rows(df)
    return df

In [ ]:
cleaned_sampled_comments = clean_data(sampled_comments)

# Count the number of rows after cleaning the data
row_count_after_cleaning = cleaned_sampled_comments.count()
print(f"Number of rows after cleaning: {row_count_after_cleaning}")

# If you want to show the DataFrame
#cleaned_sampled_comments.show()

Column author has no missing values


In [26]:
%%time
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_data_submissions = f"project/submissions/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_submissions}"
print(f"reading submissions from {s3_path}")
submissions = spark.read.parquet(s3_path, header=True)
print(f"shape of the submissions dataframe is {submissions.count():,}x{len(submissions.columns)}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading submissions from s3a://sagemaker-us-east-1-834346005548/project/submissions/yyyy=*


shape of the submissions dataframe is 95,932x68
CPU times: user 224 ms, sys: 15.6 ms, total: 239 ms
Wall time: 2min 49s


In [27]:
# check counts (ensuring all needed subreddits exist)
submissions.groupBy('subreddit').count().show()

+---------------+-----+
|      subreddit|count|
+---------------+-----+
|MakeupAddiction|95932|
+---------------+-----+



In [28]:
submissions.printSchema()

root
 |-- adserver_click_url: string (nullable = true)
 |-- adserver_imp_pixel: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- brand_safe: boolean (nullable = true)
 |-- contest_mode: boolean (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- crosspost_parent: string (nullable = true)
 |-- crosspost_parent_list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- approved_at_utc: string (nullable = true)
 |    |    |-- approved_by: string (nullable = true)
 |    |    |-- archived: boolean (nullable = true)
 |    |    |-- author: string (nullable = true)
 |    |    |-- author_flair_css_class: string (nullable = true)
 |    |    |-- author_flair_text: string (nullable = true)
 |    |    

In [29]:
# display a subset of columns
submissions.select("subreddit", "author", "title", "selftext", "created_utc", "num_comments").show()

+---------------+--------------------+--------------------+--------------------+-------------------+------------+
|      subreddit|              author|               title|            selftext|        created_utc|num_comments|
+---------------+--------------------+--------------------+--------------------+-------------------+------------+
|MakeupAddiction|  DefinitlyNotTheFBl|Boyfriend trying ...|                    |2023-01-18 23:43:31|          13|
|MakeupAddiction|           [deleted]|i’m going to a pa...|           [removed]|2023-01-18 23:44:34|           7|
|MakeupAddiction|            Readkt92|2023 Blush tracke...|                    |2023-01-18 23:46:25|           6|
|MakeupAddiction|               NoheD|How would you con...|                    |2023-01-18 23:57:36|          12|
|MakeupAddiction|      throwaway_2613|what can i do to ...|                    |2023-01-19 00:02:52|         147|
|MakeupAddiction|HuckleberryNeither63|Brow lift product...|I tried out the e...|2023-01-

In [17]:
print(f"Total number of rows: {submissions.count()}")

Total number of rows: 95932


In [18]:
print(f"Total number of unique authors: {submissions.select('author').distinct().count()}")

Total number of unique authors: 37635


In [ ]:
cleaned_submissions = clean_data(submissions)

# Count the number of rows after cleaning the data
row_count_after_cleaning_submissions = cleaned_submissions.count()
print(f"Number of rows after cleaning: {row_count_after_cleaning_submissions}")


Column adserver_click_url has 9734 missing values


Column adserver_imp_pixel has 9734 missing values
